## Protótipo Dashboard 
Neste protótipo, iremos mostrar quais dados e visualizações estarão presentes no dashboard WEB

### Import das bibliotecas

In [1]:
import pandas as pd
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker

from models import Cliente, Local, Camera, Contagem

### Conexão com o banco

In [40]:
DB_CONN_STRING = 'postgres://postgres:@localhost:5432/peoplecounter'

engine = create_engine(DB_CONN_STRING, echo=True)

### Nosso cliente cadastrado

In [58]:
query = "SELECT cliente_id, nome AS nome_cliente FROM clientes"
cliente = pd.read_sql(query, engine, index_col='cliente_id')

2018-10-23 23:47:03,894 INFO sqlalchemy.engine.base.Engine select relname from pg_class c join pg_namespace n on n.oid=c.relnamespace where pg_catalog.pg_table_is_visible(c.oid) and relname=%(name)s
2018-10-23 23:47:03,898 INFO sqlalchemy.engine.base.Engine {'name': 'SELECT cliente_id, nome AS nome_cliente FROM clientes'}
2018-10-23 23:47:03,901 INFO sqlalchemy.engine.base.Engine SELECT cliente_id, nome AS nome_cliente FROM clientes
2018-10-23 23:47:03,901 INFO sqlalchemy.engine.base.Engine {}


In [59]:
clientes

,nome
cliente_id,
1,DB SUPERMERCADOS


### Estabelecimentos do cliente cadastrados

In [82]:
query = "SELECT c.nome AS cliente_nome, l.local_id, l.nome AS local_nome, l.cep, l.endereco, l.cidade, l.estado \
        FROM locais l \
        RIGHT JOIN clientes c ON l.cliente_id = c.cliente_id \
        WHERE l.cliente_id = 1 \
        ORDER BY l.local_id"
locais = pd.read_sql(query, engine, index_col='local_id')

2018-10-23 23:59:39,647 INFO sqlalchemy.engine.base.Engine select relname from pg_class c join pg_namespace n on n.oid=c.relnamespace where pg_catalog.pg_table_is_visible(c.oid) and relname=%(name)s
2018-10-23 23:59:39,650 INFO sqlalchemy.engine.base.Engine {'name': 'SELECT c.nome AS cliente_nome, l.local_id, l.nome AS local_nome, l.cep, l.endereco, l.cidade, l.estado         FROM locais l         RIGHT JOIN clientes c ON l.cliente_id = c.cliente_id         WHERE l.cliente_id = 1         ORDER BY l.local_id'}
2018-10-23 23:59:39,654 INFO sqlalchemy.engine.base.Engine SELECT c.nome AS cliente_nome, l.local_id, l.nome AS local_nome, l.cep, l.endereco, l.cidade, l.estado         FROM locais l         RIGHT JOIN clientes c ON l.cliente_id = c.cliente_id         WHERE l.cliente_id = 1         ORDER BY l.local_id
2018-10-23 23:59:39,655 INFO sqlalchemy.engine.base.Engine {}


In [83]:
locais

,cliente_nome,local_nome,cep,endereco,cidade,estado
local_id,,,,,,
1,DB SUPERMERCADOS,DB CIDADE NOVA,69095-000,"AV. NOEL NUTELS, 1762 - CIDADE NOVA",MANAUS,AMAZONAS
2,DB SUPERMERCADOS,DB PARAIBA,69057-021,"AV. JORNALISTA HUMBERTO CALDERADO FILHO, 1.128...",MANAUS,AMAZONAS


### Cameras de cada estabelecimento do cliente cadastradas

In [84]:
query = "SELECT cl.nome AS cliente_nome, l.nome AS local_nome, cam.camera_id, cam.nome AS camera_nome \
        FROM cameras cam \
        RIGHT JOIN locais l ON cam.local_id = l.local_id \
        RIGHT JOIN clientes cl ON l.cliente_id = cl.cliente_id \
        WHERE cl.cliente_id = 1 \
        ORDER BY cam.camera_id"
cameras = pd.read_sql(query, engine, index_col='camera_id')

2018-10-23 23:59:42,049 INFO sqlalchemy.engine.base.Engine select relname from pg_class c join pg_namespace n on n.oid=c.relnamespace where pg_catalog.pg_table_is_visible(c.oid) and relname=%(name)s
2018-10-23 23:59:42,050 INFO sqlalchemy.engine.base.Engine {'name': 'SELECT cl.nome AS cliente_nome, l.nome AS local_nome, cam.camera_id, cam.nome AS camera_nome         FROM cameras cam         RIGHT JOIN locais l ON  ... (2 characters truncated) ... m.local_id = l.local_id         RIGHT JOIN clientes cl ON l.cliente_id = cl.cliente_id         WHERE cl.cliente_id = 1         ORDER BY cam.camera_id'}
2018-10-23 23:59:42,053 INFO sqlalchemy.engine.base.Engine SELECT cl.nome AS cliente_nome, l.nome AS local_nome, cam.camera_id, cam.nome AS camera_nome         FROM cameras cam         RIGHT JOIN locais l ON cam.local_id = l.local_id         RIGHT JOIN clientes cl ON l.cliente_id = cl.cliente_id         WHERE cl.cliente_id = 1         ORDER BY cam.camera_id
2018-10-23 23:59:42,054 INFO sqlalche

In [85]:
cameras

,cliente_nome,local_nome,camera_nome
camera_id,,,
1,DB SUPERMERCADOS,DB CIDADE NOVA,CAMERA ENTRADA 1
2,DB SUPERMERCADOS,DB CIDADE NOVA,CAMERA ENTRADA 2
3,DB SUPERMERCADOS,DB PARAIBA,CAM_ENTRADA_1
4,DB SUPERMERCADOS,DB PARAIBA,CAM_ENTRADA_2


### Contagem de pessoas por câmera
Considera-se que cada camera cadastrada representa uma entrada diferente para o establecimento.
Neste exemplo, veremos os dados de contagem da camera 1 do local 1 do nosso cliente cadastrado.

In [97]:
query = "SELECT cl.nome AS cliente_nome, l.nome AS local_nome, cam.camera_id, \
        cam.nome AS camera_nome, cont.contagem_id, cont.timestamp, cont.qtd_pessoas_in \
        FROM contagem cont \
        RIGHT JOIN cameras cam ON cont.camera_id = cam.camera_id\
        RIGHT JOIN locais l ON cam.local_id = l.local_id \
        RIGHT JOIN clientes cl ON l.cliente_id = cl.cliente_id \
        WHERE cl.cliente_id = 1 AND cam.camera_id = 1 \
        ORDER BY cont.contagem_id"
contagem = pd.read_sql(query, engine, index_col='contagem_id')

2018-10-24 00:46:21,723 INFO sqlalchemy.engine.base.Engine select relname from pg_class c join pg_namespace n on n.oid=c.relnamespace where pg_catalog.pg_table_is_visible(c.oid) and relname=%(name)s
2018-10-24 00:46:21,725 INFO sqlalchemy.engine.base.Engine {'name': 'SELECT cl.nome AS cliente_nome, l.nome AS local_nome, cam.camera_id,         cam.nome AS camera_nome, cont.contagem_id, cont.timestamp, cont.qtd_pess ... (156 characters truncated) ...        RIGHT JOIN clientes cl ON l.cliente_id = cl.cliente_id         WHERE cl.cliente_id = 1 AND cam.camera_id = 1         ORDER BY cont.contagem_id'}
2018-10-24 00:46:21,734 INFO sqlalchemy.engine.base.Engine SELECT cl.nome AS cliente_nome, l.nome AS local_nome, cam.camera_id,         cam.nome AS camera_nome, cont.contagem_id, cont.timestamp, cont.qtd_pessoas_in         FROM contagem cont         RIGHT JOIN cameras cam ON cont.camera_id = cam.camera_id        RIGHT JOIN locais l ON cam.local_id = l.local_id         RIGHT JOIN clientes cl O

In [98]:
contagem.head()

,cliente_nome,local_nome,camera_id,camera_nome,timestamp,qtd_pessoas_in
contagem_id,,,,,,
1,DB SUPERMERCADOS,DB CIDADE NOVA,1,CAMERA ENTRADA 1,2018-10-24 00:43:03.003574,2
2,DB SUPERMERCADOS,DB CIDADE NOVA,1,CAMERA ENTRADA 1,2018-10-24 00:43:06.017777,1
3,DB SUPERMERCADOS,DB CIDADE NOVA,1,CAMERA ENTRADA 1,2018-10-24 00:43:09.007212,4
4,DB SUPERMERCADOS,DB CIDADE NOVA,1,CAMERA ENTRADA 1,2018-10-24 00:43:12.059917,5
5,DB SUPERMERCADOS,DB CIDADE NOVA,1,CAMERA ENTRADA 1,2018-10-24 00:43:15.034008,6


### Resumo Estatístico

In [106]:
resumo = pd.DataFrame(contagem.qtd_pessoas_in.describe())
resumo

,qtd_pessoas_in
count,9.000000
mean,3.000000
std,1.936492
min,0.000000
25%,2.000000
50%,3.000000
75%,4.000000
max,6.000000
